In [1]:
import json
from pprint import pprint
from matplotlib import pyplot as plt
import cv2
import copy
import operator
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import os
%matplotlib qt

In [2]:
def readJson(file):
    with open(file, 'r') as fp:
        data = json.load(fp)
        
    return data

In [3]:
def read_jsons(folder):
    jsons = [json for json in os.listdir(folder) if json.endswith(".json")]
    
    return jsons

In [14]:
def data_points(jsons, json_folder_dir):
    data_points = []
    
    for i in range(0, len(jsons)):
        data_points.append([])
        with open((json_folder_dir + jsons[i])) as f:
            data = json.load(f)
        
        n_people = len(data['people'])
    
        for j in range(0, n_people):
            data_points[i].append([])
            datas = data['people'][j]['pose_keypoints_2d']
        
            for k in range(0, len(datas)):
                data_points[i][j].append(datas[k])
                
    return data_points

In [15]:
def data_classification(data):
    body_label = {0:"Nose", 1:"Neck", 2:"RShoulder", 3:"RElbow", 4:"RWrist", 5:"LShoulder", 6:"LElbow", 7:"LWrist", 8:"MidHip", 9:"RHip", 10:"RKnee", 11:"RAnkle", 12:"LHip", 13:"LKnee",14:"LAnkle", 15:"REye", 16:"LEye", 17:"REar", 18:"LEar", 19:"LBigToe", 20:"LSmallToe", 21:"LHeel", 22:"RBigToe", 23:"RSmallToe", 24:"RHeel"}
    parts = {}
    people = {}
    
    for person in range(0, len(data)):
        points = data[person]
        for i in range(0, len(points)):
            parts[body_label[i]] = points[i]
        
        people[person] = parts
        parts = {}
        
    return people

In [58]:
def organise_data(data, keyStart):
    total = len(data)
    people_body_point = {}
    
    for i in range(0, total):
        people = len(data[i])
        frame_points = {}
        id_number = 0
        if people != 0:
            for j in range(0, people):
                points = data[i][j]
                n_points = len(points)
                body_points = []
                data_certainty = 0
                for k in range(0, n_points - 1, 3):
                    body_points.append((points[0 + k], points[1 + k]))
                    data_certainty += points[2 + k]
                    
                
                data_certainty = data_certainty / 26
                
                if data_certainty >= 0.5:
                    frame_points[id_number] = body_points
                    id_number += 1

            
            organised = data_classification(frame_points)
            people_body_point[i + keyStart] = organised
        else:
            print("No person")
        
    return people_body_point

In [56]:
def read_folders(superFolder):
    subFolders = next(os.walk('Sports_Skeleton/' + superFolder + '/.'))[1]
    
    keyStart = 0
    
    total = {}
    
    for i in range(len(subFolders)):
        json_folder = 'Sports_Skeleton/' + superFolder + '/' + subFolders[i]
        json_folder_slash = json_folder + '/'
        
        jsons = read_jsons(json_folder)
        
        data_points1 = data_points(jsons, json_folder_slash)
        
        part_points = organise_data(data_points1, keyStart)
        
        total.update(part_points)
        
        print("Value of first key is " + str(list(part_points.keys())[0]))
        print("Value of keyStart is " + str(keyStart))
        
        keyStart += len(data_points1)
        
        print("Updated value of keyStart is " + str(keyStart))
        
    return total

In [3]:
def section_separator(data, start, end):
    
    exercise_one = {}
    exercise_two = {}
    
                
    for i in range(start, end + 1):
        
        for j in data[str(i)]:
            if j == '0':
                exercise_one[i] = data[str(i)][j]
            else:
                exercise_two[i] = data[str(i)][j]
        
#         i += 1

    return exercise_one, exercise_two

In [70]:
def organiseAllPoints(data):
    new_data = {}
    
    # Initialisation
    for keys in data[list(data.keys())[0]]:
        
        new_data[keys] = []
       
    for keys in data:
        for Lkeys in data[keys]:
            coordinate = new_data[Lkeys]
            
            points = data[keys][Lkeys]
            
            if points[0] != 0 and points[1] != 0:
                previousPoints = copy.deepcopy(points)
                coordinate.append(points)
            else:
                coordinate.append(previousPoints)
    
            
            new_data[Lkeys] = coordinate            
    
    return new_data

In [72]:
def rescale(data):
    
    allPoints = organiseAllPoints(data)
    
    new_data = {}
    
    for keys in allPoints:
#         ini = {}
#         ini['x'] = []
#         ini['y'] = []
        
        new_data[keys] = []
        
    fit_points = []
    first = True
    
    for keys in allPoints:
        if first:
            fit_points = allPoints[keys]
            first = False
        else:
            fit_points + allPoints[keys]
            
    scaler = StandardScaler().fit(fit_points)
    
    for keys in allPoints:
        points = scaler.transform(allPoints[keys])
        
#         x = points[:, 0]
#         y = points[:, 1]
#         x_mean = np.mean(x)
#         x_std = np.std(x)
#         y_mean = np.mean(y)
#         y_std = np.std(y)
        
#         X = [x_mean, x_std]
#         Y = [y_mean, y_std]
        
#         new_data[keys]['x'] = X
#         new_data[keys]['y'] = Y

        new_data[keys] = points
        
    return new_data

In [8]:
# section_player("DS3.MOV", ds4_data, 3580, 4370)
# 210, 980
# 1430, 2210
#2550, 3330
#3580, 4370

In [59]:
len(read_folders('Squats'))

Value of first key is 0
Value of keyStart is 0
Updated value of keyStart is 554
Value of first key is 554
Value of keyStart is 554
Updated value of keyStart is 906
Value of first key is 906
Value of keyStart is 906
Updated value of keyStart is 3479
Value of first key is 3479
Value of keyStart is 3479
Updated value of keyStart is 5239
Value of first key is 5239
Value of keyStart is 5239
Updated value of keyStart is 7505
Value of first key is 7505
Value of keyStart is 7505
Updated value of keyStart is 10048
Value of first key is 10048
Value of keyStart is 10048
Updated value of keyStart is 13720
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person
No person

18073

In [9]:
squat1, arm1 = section_separator(ds2_data, 135, 800)
arm2, squat2 = section_separator(ds2_data, 1230, 1910)
squat3, arm3 = section_separator(ds2_data, 2700, 3350)
arm4, squat4 = section_separator(ds2_data, 3600, 4280)

In [10]:
squat5, arm5 = section_separator(ds3_data, 210, 930)
arm6, squat6 = section_separator(ds3_data, 1250, 1990)
squat7, arm7 = section_separator(ds3_data, 2330, 2980)
arm8, squat8 = section_separator(ds3_data, 3725, 4350)
squat9, arm9 = section_separator(ds3_data, 4585, 5325)

In [11]:
squat10, starJump1 = section_separator(ds4_data, 210, 980)
starJump2, squat11 = section_separator(ds4_data, 1430, 2210)
squat12, starJump2 = section_separator(ds4_data, 2550, 3330)
starJump3, squat13 = section_separator(ds4_data, 3580, 4370)

In [12]:
x = np.linspace(210, 980, 980 - 210)

In [73]:
arm1_rescale = rescale(arm1)
arm2_rescale = rescale(arm2)
arm3_rescale = rescale(arm3)
arm4_rescale = rescale(arm4)
arm5_rescale = rescale(arm5)
arm6_rescale = rescale(arm6)
arm7_rescale = rescale(arm7)
arm8_rescale = rescale(arm8)
arm9_rescale = rescale(arm9)
squat1_rescale = rescale(squat1)
squat2_rescale = rescale(squat2)
squat3_rescale = rescale(squat3)
squat4_rescale = rescale(squat4)
squat5_rescale = rescale(squat5)
squat6_rescale = rescale(squat6)
squat7_rescale = rescale(squat7)
squat8_rescale = rescale(squat8)
squat9_rescale = rescale(squat9)
squat10_rescale = rescale(squat10)
squat11_rescale = rescale(squat11)
squat12_rescale = rescale(squat12)
squat13_rescale = rescale(squat13)

{'Nose': array([[-0.05545211,  0.18230149],
        [ 0.06712273, -0.10116615],
        [ 0.06920604, -0.38501965],
        ...,
        [-1.24378611,  0.03764955],
        [-1.24238307, -0.24094651],
        [-1.1202759 , -0.24113945]]),
 'Neck': array([[ 0.63106056,  3.86738079],
        [ 0.6310393 ,  3.73633066],
        [ 0.63082672,  3.73464249],
        ...,
        [-0.87212881,  3.87992147],
        [-0.87112967,  3.87905327],
        [-0.86845114,  3.8789568 ]]),
 'RShoulder': array([[-0.68042226,  3.87972853],
        [-0.61998516,  3.87480873],
        [-0.68137888,  3.87418169],
        ...,
        [-2.30595263,  4.3013366 ],
        [-2.24676977,  4.44111697],
        [-2.18161335,  4.44126167]]),
 'RElbow': array([[-1.36937962,  8.13507121],
        [-1.43349439,  7.70497406],
        [-1.55725969,  7.14373033],
        ...,
        [-2.49585223,  8.70215118],
        [-2.43377825,  8.56130968],
        [-2.37268215,  8.69940188]]),
 'RWrist': array([[-0.99434226, 10.40